# Finding our best performing salespeople and products

## Introduction

**Business Context.** You work for AdventureWorks, a company that sells outdoor sporting equipment. The company has many different locations and has been recording the sales of different locations on various products. You, their new data scientist, has been tasked with the question "what are our best products and sales people and how can use improve our other products and sales people with this information?"

You have been given access to their database with documentation from the IT department. Your job is to extract meaninful insights from this database to help increase sales. First, you will look at the best products and try to see how different products behave in different categories. Second, you will analyze the best sales people to see if commission percentage motivates them to sell more.

**Business Problem.** Your task is to query the companies database and perform simple satistical analys directly using ```SQL```.

**Analytical Context.** You are given a database (stored in the ```data``` folder) containing all the databases for the company. To simulate this in a real world setting, we will first create the database in the cloud by:

1. Creating an RDS instance on AWS to store a PostgreSQL database
2. Uploading the database using the ```install.sql``` file provided

The company has been pretty vague about how they expect you to extract insights, but you have come up with the following plan of attack:

1. Look at how product ratings and total sales are related
2. See how products sell in different subcategories (bikes, helmets, socks, etc.)
3. Calculating which sales people have performed the best in the past year
4. Seeing if total sales are correlated with their commission percentage

Of course, this is only your initial plan of attack. As you explore the database, your strategy will change.

## Setting up AWS

For this case, we will be simulating as if the company has its data in the cloud and will be doing everything through the terminal using `psql`.

### Question (15 min):

Repeat the steps in case 12.3 to create a new RDS instance with a Postgres database.

## Overview of the data

The data for the course is contained in the ```./data``` directory but it is the ```AdventureWorks``` sample database provided by Microsoft. We will be focusing on the ```Sales``` and ```Production``` schemas. A brief overview of our tables of interest is presented here but a complete documentation can be found [here](https://dataedo.com/download/AdventureWorks.pdf). The tables you should pay particular attention to (and look at in the documentation) are:

- **Production.Product**
- **Production.ProductReview**
- **Production.ProductModelProductDescriptionCulture**
- **Production.ProductDescription**
- **Sales.SalesPerson**
- **Sales.SalesOrderHeader**
- **Sales.SalesOrderDetail**

Regardless, when doing the exercsises, **you should constantly refer to the documentation above**. You should also focus on building your queries one step at a time. If you have 3 ```JOIN```'s to make in a query, you should start with the first ```SELECT``` and perform the ```JOIN```'s one at a time and output them to make sure they are doing the correct thing.

### Question (15 min):

Upload the database to your server using the following two commands. The first will create the database and the second will upload the database. **Note:** you will have to be in the folder containing the case. After it is uploaded explore the database by performing some basic queries to get a better sense of the database. At the end, write a query to extract the first 5 rows of the table containing the names of the product descriptions. **Note:** you should be able to replace the "+" with "|" in your ```psql``` output to format it nicely as a Markdown table.

```bash
psql -c "CREATE DATABASE \"Adventureworks\";"
psql -d Adventureworks < install.sql
```

**Answer**

```sql
SELECT * FROM Production.ProductDescription LIMIT 5;
```

productdescriptionid |                                                                       description                                                                       |               rowguid                |    modifieddate     
----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------|---------------------
                    3 | Chromoly steel.                                                                                                                                         | 301eed3a-1a82-4855-99cb-2afe8290d641 | 2013-04-30 00:00:00
                    4 | Aluminum alloy cups; large diameter spindle.                                                                                                            | dfeba528-da11-4650-9d86-cafda7294eb0 | 2013-04-30 00:00:00
                    5 | Aluminum alloy cups and a hollow axle.                                                                                                                  | f7178da7-1a7e-4997-8470-06737181305e | 2013-04-30 00:00:00
                    8 | Suitable for any type of riding, on or off-road. Fits any budget. Smooth-shifting with a comfortable ride.                                              | 8e6746e5-ad97-46e2-bd24-fcea075c3b52 | 2013-04-30 00:00:00
                   64 | This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road. | 7b1c4e90-85e2-4792-b47b-e0c424e2ec94 | 2013-04-30 00:00:00

## Finding our best sellers

The company is curious which of their products is the most popular among the customers. You first idea is to see if the average rating given in reviews is correlated with the number of sales in that product. This will help guide future decision making.

### Exercise 1 (15 min):

Using the ```Production.Product``` and ```Production.ProductReview``` tables, ```JOIN``` them and rank the product according to their average review rating. What are the names and ID's of the top products? Limit results to 5 rows and order by decreasing rating.

```sql
SELECT p.ProductID, p.Name, Reviews.AvgRating
FROM Production.Product p
INNER JOIN
(
    SELECT ProductID, AVG(Rating) as AvgRating
    FROM Production.ProductReview
    GROUP BY ProductID
) Reviews
ON Reviews.ProductID = p.ProductID
LIMIT 5;
```

 productid |          name          |     avgrating      
-----------|------------------------|--------------------
       709 | Mountain Bike Socks, M | 5.0000000000000000
       798 | Road-550-W Yellow, 40  | 5.0000000000000000
       937 | HL Mountain Pedal      | 3.0000000000000000

### Exercise 2 (30 min)

Much to your disappointment, there are only 3 products with ratings. This is not nearly enough to perform a correlation between reviews and total sales. However, your manager wants a description of the highest rated product (the socks) for an upcoming sale. You will have to extract the product description from ```Production.ProductDescription``` by using multiple tables. It's ok to hard code the ProductID as this is a one-off. Note, you will have to start off with the ProductModelID in the ```Production.Product``` table. Then go through the ```Production.ProductModelProductDescriptionCulture```. There are multiple product models, but we only want the oldest one (they are going on sale). Finally, use this ProductDescriptionID to get the actual description from ```Production.ProductDescription```. Use the documentation provided above if you need help navigating the structure!

```sql
SELECT Description FROM
(
    SELECT MIN(ProductDescriptionID) as ProductDescriptionID
    FROM
    (
        SELECT ProductModelID
        FROM Production.Product
        WHERE ProductID = 709
    ) p
    INNER JOIN Production.ProductModelProductDescriptionCulture pmpdc
    ON p.ProductModelID = pmpdc.ProductModelID
) a
INNER JOIN Production.ProductDescription pd
ON a.ProductDescriptionID = pd.ProductDescriptionID
```

**Description:** Combination of natural and synthetic fibers stays dry and provides just the right cushioning.

### Exercise 3 (30 min)

Since we cannot infer the most popular from the reviews, we will calculate the top sellers by the number of items sold for each product in each product subcategory. Using the ```Sales.SalesOrderDetail``` table, calculate the number of each product sold (don't forget OrderQty!). Then you will to merge with ```Production.Product``` to get the subcategory. The final output should be the products in decreasing order with their product and subcategory name. Limit results to 5 rows.

```sql
SELECT ps.Name as SubcategoryName, QuantityWithSubcategory.Name, QuantityWithSubcategory.Quantity
FROM
(
    SELECT Quantity.Quantity, p.ProductSubcategoryID, p.Name
    FROM
    (
        SELECT ProductID, SUM(OrderQty) as Quantity
        FROM Sales.SalesOrderDetail
        GROUP BY ProductID
    ) Quantity
    INNER JOIN Production.Product p
    ON p.ProductID = Quantity.ProductID
) QuantityWithSubcategory
INNER JOIN Production.ProductSubcategory ps
ON QuantityWithSubcategory.ProductSubcategoryID = ps.ProductSubcategoryID
ORDER BY Quantity DESC
LIMIT 5;
```

  subcategoryname  |            name            | quantity 
-------------------|----------------------------|----------
 Caps              | AWC Logo Cap               |     8311
 Bottles and Cages | Water Bottle - 30 oz.      |     6815
 Helmets           | Sport-100 Helmet, Blue     |     6743
 Jerseys           | Long-Sleeve Logo Jersey, L |     6592
 Helmets           | Sport-100 Helmet, Black    |     6532

### Exercise 4 (30 min)

We are going to correlate quantity sold with the price of each item in each subcategory. Alter the previous query to include the price of the item. At the end, group by subcategory and perform a correlation on quantity and price using the ```corr()``` function.

```sql
SELECT a.SubcategoryName, corr(a.Quantity, a.ListPrice) AS correlation
FROM
(
    SELECT ps.Name as SubcategoryName, QuantityWithSubcategory.Name, QuantityWithSubcategory.Quantity, QuantityWithSubcategory.ListPrice
    FROM
    (
        SELECT Quantity.Quantity, p.ProductSubcategoryID, p.Name, p.ListPrice
        FROM
        (
            SELECT ProductID, SUM(OrderQty) as Quantity
            FROM Sales.SalesOrderDetail
            GROUP BY ProductID
        ) Quantity
        INNER JOIN Production.Product p
        ON p.ProductID = Quantity.ProductID
    ) QuantityWithSubcategory
    INNER JOIN Production.ProductSubcategory ps
    ON QuantityWithSubcategory.ProductSubcategoryID = ps.ProductSubcategoryID
    ORDER BY Quantity DESC
) a
GROUP BY a.SubcategoryName
```

  subcategoryname  |      correlation      
-------------------|-----------------------
 Bib-Shorts        |                      
 Bike Racks        |                      
 Bike Stands       |                      
 Bottles and Cages |   -0.9701687537690749
 Bottom Brackets   |                     1
 Brakes            |                      
 Caps              |                      
 Chains            |                      
 Cleaners          |                      
 Cranksets         |   0.49044151769746913
 Derailleurs       |   -0.9999999999999999
 Fenders           |                      
 Forks             |                     1
 Gloves            |  -0.13288344614071643
 Handlebars        | -0.011298492617637799
 Headsets          |    0.5329906098447924
 Helmets           |                      
 Hydration Packs   |                      
 Jerseys           |  -0.30328486794255455
 Locks             |                      
 Mountain Bikes    |    0.3361259034857538
 Mountain Frames   |   0.20387689301243794
 Pedals            |  -0.20081610268718958
 Pumps             |                      
 Road Bikes        |   -0.4212668912569426
 Road Frames       |   -0.3437128937002691
 Saddles           |   0.45558597441130877
 Shorts            |    0.7650704777844313
 Socks             |   -0.8234535670797966
 Tights            |                      
 Tires and Tubes   |   -0.7919373126860129
 Touring Bikes     |   0.36225766397408676
 Touring Frames    |   0.33835632563248563
 Vests             |                      
 Wheels            |   -0.2668638687651112

Notice that some subcategories are degenerate. We expect to see groups like this when they don't have many products. There also seems to be a mixture of correlation coefficients. Looking at this table, we see that socks are negatively correlated which makes sense as they are a substitutable good. However, the touring and mountain bikes have a positive coefficient. Possibly due to the fact that people tend to be in those markets looking for a higher end product.

## Finding our top performing sales people

As mentioned earlier, we want to see which of our sales are the top performing and whether or not we can incentivize them in an appropriate manner. Mainly, we want to see if the commission percentage we give sales people motivate them to sell more.

### Exercise 5 (15 min):

Query the tables in the ```Sales``` schema to return a table of the sales person ordered by how much they sold this year (YTD). Only show the first 5 rows.

```sql
SELECT BusinessEntityID, SalesYTD FROM Sales.SalesPerson ORDER BY SalesYTD DESC LIMIT 5;
```

 businessentityid |   salesytd   
------------------|--------------
              276 | 4251368.5497
              289 | 4116871.2277
              275 | 3763178.1787
              277 | 3189418.3662
              290 | 3121616.3202

### Exercise 6 (15 min):

You are skeptical of these sales numbers as it is early on in the year. So you want to go through and validate these by computing the sales for each given sales person using other tables. Using  ```SalesOrderHeader``` (there is a column SubTotal) in the ```Sales``` schema, calculate the sales for each sales person for **this past year** (2014).  Sales that do not have an associated sales person is assigned an ID of 0. You should exclude this from the results. Limit results to 5 rows and order by descending total sales.

```sql
SELECT SalesPersonID, SUM(SubTotal) AS TotalSales
FROM Sales.SalesOrderHeader soh
WHERE soh.OrderDate >= make_date(2014, 1, 1) AND soh.SalesPersonID != 0
GROUP BY SalesPersonID
ORDER BY TotalSales DESC
LIMIT 5;
```

 salespersonid |  totalsales  
---------------|--------------
           289 | 1382996.5839
           276 | 1271088.5216
           275 | 1057247.3786
           282 | 1044810.8277
           277 | 1040093.4071

We see right away that there are descrepencies between the two sales totals. For the remainder of the exercise you will use your answer from exercise 6 as your are told the SalesYTD in the SalesPerson table is for the company's fiscal year and not the calendar year.

### Exercise 7 (30 min):

Looking at the documentation, you see that SubTotal in the ```SalesOrderHeader``` table is calculated from other tables in the database. Let us repeat the previous exercise except calculate SubTotal manually. Using ```SalesOrderDetail``` and ```SalesOrderHeader``` in the ```Sales``` schema, calculate the sales for each sales person for **this past year** (2014). **Hint:** You will have to ```JOIN``` ```SalesOrderDetail``` on ```SalesOrderHeader``` to get the sales person, calculate line totals for each sale using appropriate discounts, then sum all the line totals to get the total sale. Limit results to 5 rows and order by descending total sales. You will want to use ```WITH ... AS``` to save subqueries as we will assume that SubTotal doesn't exist moving forward.

```sql
WITH Orders AS (
    SELECT SalesOrderID, SUM(UnitPrice*(1-UnitPriceDiscount)*OrderQty) as OrderTotal
    FROM Sales.SalesOrderDetail
    GROUP BY SalesOrderID
),

SalesPersonTotalSales AS (
    SELECT SalesPersonID, SUM(OrderTotal) as TotalSales
    FROM Orders o
    INNER JOIN Sales.SalesOrderHeader soh ON o.SalesOrderID = soh.SalesOrderID
    WHERE soh.OrderDate >= make_date(2014, 1, 1) AND soh.SalesPersonID != 0
    GROUP BY SalesPersonID
)

SELECT *
FROM SalesPersonTotalSales
ORDER BY TotalSales DESC
LIMIT 5;
```

 salespersonid |   totalsales   
---------------|----------------
           289 | 1382996.583910
           276 | 1271088.521461
           275 | 1057247.378572
           282 | 1044810.827687
           277 | 1040093.406901

### Exercise 8 (30 min):

The final step of your initial analysis of finding whether or not commision percentage impacts total sales is to see their correlation. Using ```corr```, see if there is a positive relationship between total sales and commision percentage. **Optional:** If you regress on comission percentage to total sales, the slope will be massive, so normalize by the average of the total sales if you want to see the extent of the relationship.

**Answer**

```sql
WITH Orders AS (
    SELECT SalesOrderID, SUM(UnitPrice*(1-UnitPriceDiscount)*OrderQty) as OrderTotal
    FROM Sales.SalesOrderDetail
    GROUP BY SalesOrderID
),

SalesPersonTotalSales AS (
    SELECT SalesPersonID, SUM(OrderTotal) as TotalSales
    FROM Orders o
    INNER JOIN Sales.SalesOrderHeader soh ON o.SalesOrderID = soh.SalesOrderID
    GROUP BY SalesPersonID
)
 
SELECT corr(spts.TotalSales, sp.CommissionPct) as correlation
FROM SalesPersonTotalSales spts
JOIN Sales.SalesPerson sp 
ON sp.BusinessEntityID = spts.SalesPersonID;
```

$correlation=0.4377704110963032$

### Exercise 9 (30 min):

You now just realized that your company is multinational and has offices in different countries around the world. Furthermore, you are told the sales are recorded in the local currency so you must convert it to USD if you wish to compare the different sales people. Construct a new ```Sales.SalesPerson``` table to include a new column for ```CurrencyCode```. You will have to do this by extracting the TerritoryID of the sales person from ```Sales.SalesTerritory``` then using CountryRegionCode in that table to get CurrencyCode from ```Sales.CountryRegionCurrency```. This will involve multiple ```JOIN```'s. Limit results to 5 rows.

**Answer**

```sql
WITH SalesPersonWithCurrency AS (
    
    SELECT a.BusinessEntityID, crc.CurrencyCode FROM
    (
        SELECT sp.BusinessEntityID, st.CountryRegionCode
        FROM Sales.SalesPerson sp
        INNER JOIN Sales.SalesTerritory st
        ON sp.TerritoryID = st.TerritoryID
    ) a
    INNER JOIN Sales.CountryRegionCurrency crc ON crc.CountryRegionCode = a.CountryRegionCode
)

SELECT * FROM SalesPersonWithCurrency
LIMIT 5;
```

 businessentityid | currencycode 
------------------|--------------
              275 | USD
              276 | USD
              277 | USD
              278 | CAD
              279 | USD

### Exercise 10 (45 min):

Now that we have the currency codes associated with each sales person, redo exercise 7 to take the currency exchange into account. **Hint:** the rates in the```Sales.CurrencyRate``` table always goes from FromCurrencyCode='USD' to ToCurrencyCode='Desired Currency Code' and they are listed every day. When calculating line totals, use the AverageRate for that day. You should be able to reuse a lot of exercise 8.

**Answer**

```sql
WITH Orders AS (
    SELECT SalesOrderID, SUM(UnitPrice*(1-UnitPriceDiscount)*OrderQty) as OrderTotal
    FROM Sales.SalesOrderDetail
    GROUP BY SalesOrderID
),

SalesPersonWithCurrency AS (
    SELECT a.BusinessEntityID, crc.CurrencyCode FROM
    (
        SELECT sp.BusinessEntityID, st.CountryRegionCode
        FROM Sales.SalesPerson sp
        INNER JOIN Sales.SalesTerritory st
        ON sp.TerritoryID = st.TerritoryID
    ) a
    INNER JOIN Sales.CountryRegionCurrency crc ON crc.CountryRegionCode = a.CountryRegionCode
),

OrdersWithCurrency AS (
    SELECT a.SalesPersonID, a.OrderTotal, a.OrderDate, spwc.CurrencyCode FROM
    (
        SELECT *
        FROM Orders o
        INNER JOIN Sales.SalesOrderHeader soh ON o.SalesOrderID = soh.SalesOrderID
        WHERE soh.OrderDate >= make_date(2014, 1, 1) AND soh.SalesPersonID != 0
    ) a
    INNER JOIN SalesPersonWithCurrency spwc ON spwc.BusinessEntityID = a.SalesPersonID
),

OrdersWithCurrencyRate AS (
    SELECT owc.SalesPersonID, owc.OrderTotal, owc.OrderTotal/cr.AverageRate AS OrderTotalAdjusted, owc.OrderDate, owc.CurrencyCode, cr.AverageRate
    FROM OrdersWithCurrency owc
    INNER JOIN Sales.CurrencyRate cr ON cr.ToCurrencyCode = owc.CurrencyCode
    WHERE cr.CurrencyRateDate = owc.OrderDate
),

SalesPersonTotalSalesAdjusted AS (
    SELECT SalesPersonID, SUM(OrderTotalAdjusted) AS TotalSalesAdjusted
    FROM OrdersWithCurrencyRate
    GROUP BY SalesPersonID
)

SELECT *
FROM SalesPersonTotalSalesAdjusted
ORDER BY TotalSalesAdjusted DESC
LIMIT 5;
```

 salespersonid |    totalsalesadjusted    
---------------|--------------------------
           289 | 2146418.4023948001053897
           276 | 1271088.5214610000000000
           275 | 1057247.3785720000000000
           277 | 1040093.4069010000000000
           290 |  844392.7295821792961138

### Exercise 11 (15 min):

How does the correlation change from exercise 8 once you've adjusted for the currency?

```sql
WITH Orders AS (
    SELECT SalesOrderID, SUM(UnitPrice*(1-UnitPriceDiscount)*OrderQty) as OrderTotal
    FROM Sales.SalesOrderDetail
    GROUP BY SalesOrderID
),

SalesPersonWithCurrency AS (
    SELECT a.BusinessEntityID, crc.CurrencyCode FROM
    (
        SELECT sp.BusinessEntityID, st.CountryRegionCode
        FROM Sales.SalesPerson sp
        INNER JOIN Sales.SalesTerritory st
        ON sp.TerritoryID = st.TerritoryID
    ) a
    INNER JOIN Sales.CountryRegionCurrency crc ON crc.CountryRegionCode = a.CountryRegionCode
),

OrdersWithCurrency AS (
    SELECT a.SalesPersonID, a.OrderTotal, a.OrderDate, spwc.CurrencyCode FROM
    (
        SELECT *
        FROM Orders o
        INNER JOIN Sales.SalesOrderHeader soh ON o.SalesOrderID = soh.SalesOrderID
        WHERE soh.OrderDate >= make_date(2014, 1, 1) AND soh.SalesPersonID != 0
    ) a
    INNER JOIN SalesPersonWithCurrency spwc ON spwc.BusinessEntityID = a.SalesPersonID
),

OrdersWithCurrencyRate AS (
    SELECT owc.SalesPersonID, owc.OrderTotal, owc.OrderTotal/cr.AverageRate AS OrderTotalAdjusted, owc.OrderDate, owc.CurrencyCode, cr.AverageRate
    FROM OrdersWithCurrency owc
    INNER JOIN Sales.CurrencyRate cr ON cr.ToCurrencyCode = owc.CurrencyCode
    WHERE cr.CurrencyRateDate = owc.OrderDate
),

SalesPersonTotalSalesAdjusted AS (
    SELECT SalesPersonID, SUM(OrderTotalAdjusted) AS TotalSalesAdjusted
    FROM OrdersWithCurrencyRate
    GROUP BY SalesPersonID
)

SELECT corr(sptsa.TotalSalesAdjusted, sp.CommissionPct) as correlation
FROM SalesPersonTotalSalesAdjusted sptsa
JOIN Sales.SalesPerson sp 
ON sp.BusinessEntityID = sptsa.SalesPersonID;
```

$correlation = 0.37347349625789983$

We see that correlation has gone down which indicates that offering a higher commission is less important to motivate sales people as before but still has a positive relationship nonetheless.

## Conclusion

Over the course of the case, we explored a database that started off completely foreign to us by starting out by looking at the documentation and then constructing simple queries to get a feel of the database. We then built on these queries ```JOIN```ing them across many tables to be able to perform some basically statistical analysis. We found both unsurprising (commission correlated with total sales) and surprising (different subcategories have different +/- correlation between total sales of products and price). However, the main skill you should takeway is seeing how databases are normally structured and getting practice navigating a new dataset using only the documentation provided.